In [ ]:
import os
import pandas as pd
import numpy as np
import numpy 
import sys
from collections import *
pd.options.mode.chained_assignment = None
import random
from tqdm import tqdm

In [6]:
os.chdir('/Users/DennisLin/Documents/Python/CSE258/final/kkbox_recommender/')
data_path = './kkbox-music-recommendation-challenge/'

In [87]:
ls kkbox-music-recommendation-challenge/

members.csv               song_extra_info.csv       test.csv
members.csv.7z            song_extra_info.csv.7z    test.csv.7z
sample_submission.csv     songs.csv                 train.csv
sample_submission.csv.7z  songs.csv.7z              train.csv.7z


### Members

In [12]:
class Members():
    def __init__(self, path):
        self.members = pd.read_csv(path)
        self.preprocess()
    
    def preprocess(self):
        '''
        1. remove wierd age
        2. replace registers -> years
        3. add using date
        4. gender to three class
        '''
        #print info
        self.info()
        
        # 1
        wierd_age = list()
        for age, count in Counter(self.members['bd']).items():
            if count < 10:
                wierd_age.append(age)
        for drop_age in wierd_age:
            self.members = self.members[self.members['bd'] != drop_age]        
        
        #2 & 3
        self.members['registration_init_time'] = pd.to_datetime(self.members['registration_init_time'], format='%Y%m%d')
        self.members['expiration_date'] = pd.to_datetime(self.members['expiration_date'], format='%Y%m%d')
        self.members = self.members[self.members['expiration_date'] != '1970-01-01 00:00:00']
        self.members['activation_days'] = self.members['expiration_date'] - self.members['registration_init_time']
        self.members['activation_days'].astype(int)
        self.members['expiration_date'] = pd.DatetimeIndex(self.members['expiration_date']).year
        self.members['expiration_date'] -= min(self.members['expiration_date'])
        self.members['registration_init_time'] = pd.DatetimeIndex(self.members['registration_init_time']).year
        self.members['registration_init_time'] -= min(self.members['registration_init_time'])


        
        #4    
        self.members['gender'][self.members['gender'].isnull()] = 0
        self.members['gender'][self.members['gender'] == 'male'] = 1
        self.members['gender'][self.members['gender'] == 'female'] = -1
        
        #print info
        self.info()


    
    def info(self):
        print("The number of total users is: {}".format(len(self.members)))


In [15]:
members = Members(os.path.join(data_path, 'members.csv'))

In [70]:
members.members

,msno,city,bd,gender,registered_via,registration_init_time,expiration_date,activation_days
0,XQxgAYj3klVKjR3oxPPXYYFp4soD4TuBghkhMTD4oTw=,1,0,0,7,7,13,2223 days
1,UizsfmJb9mV54qE9hCYyU07Va97c0lCRLEQX3ae+ztM=,1,0,0,7,11,13,725 days
2,D8nEhsIOBSoE6VthTaqDX8U6lqjJ7dLdr72mOyLya2A=,1,0,0,4,12,13,457 days
3,mCuD+tZ1hERA/o5GPqk38e041J8ZsBaLcu7nGoIIvhI=,1,0,0,9,11,11,1 days
4,q4HRBfVSssAFS9iRfxWrohxuk9kCYMKjHOEagUMV6rQ=,1,0,0,4,13,13,138 days
...,...,...,...,...,...,...,...,...
34398,Wwd/cudKVuLJ3txRVxlg2Zaeliu+LRUfiBmfrnxhRCY=,1,0,0,7,9,13,1399 days
34399,g3JGnJX6Hg50lFbrNWfsHwCUmApIkiv2M8sXOaeXoIQ=,4,18,1,3,10,13,937 days
34400,IMaPMJuyN+ip9Vqi+z2XuXbFAP2kbHr+EvvCNkFfj+o=,1,0,0,7,9,13,1498 days
34401,WAnCAJjUty9Stv8yKtV7ZC7PN+ilOy5FX3aIJgGPANM=,1,0,0,7,11,13,701 days


### Songs

In [79]:
songs = pd.read_csv(os.path.join(data_path, "songs.csv"))

In [80]:
songs

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
1,o0kFgae9QtnYgRkVPqLJwa05zIhRlUjfF7O1tDw0ZDU=,197328,444,BLACKPINK,TEDDY| FUTURE BOUNCE| Bekuh BOOM,TEDDY,31.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0
...,...,...,...,...,...,...,...
2296315,lg6rn7eV/ZNg0+P+x77kHUL7GDMfoL4eMtXxncseLNA=,20192,958,Catherine Collard,Robert Schumann (1810-1856),NaN,-1.0
2296316,nXi1lrSJe+gLoTTNky7If0mNPrIyCQCLwagwR6XopGU=,273391,465,紀文惠 (Justine Chi),NaN,NaN,3.0
2296317,9KxSvIjbJyJzfEVWnkMbgR6dyn6d54ot0N5FKyKqii8=,445172,1609,Various Artists,NaN,NaN,52.0
2296318,UO8Y2MR2sjOn2q/Tp8/lzZTGKmLEvwZ20oWanG4XnYc=,172669,465,Peter Paul & Mary,NaN,NaN,52.0


In [71]:
songs['artist_name'] = songs.groupby('artist_name').ngroup()

In [82]:
songs.groupby('genre_ids').size()

genre_ids
1000                89
1000|2154|751        6
1007               693
1011             34013
1011|2189|367        4
                 ...  
979               1295
986                577
986|1955             3
993                  3
993|751            578
Length: 1045, dtype: int64

In [68]:
valid_artists = set()
threshold = 10
for singer, count in songs.groupby("artist_name").size().items():
    if count > threshold:
        valid_artists.add(singer)
songs[songs['artist_name'].isin(valid_artists)]

,song_id,song_length,genre_ids,artist_name,composer,lyricist,language
0,CXoTN1eb7AI+DntdU1vbcwGRV4SCIDxZu+YD8JP8r4E=,247640,465,張信哲 (Jeff Chang),董貞,何啟弘,3.0
2,DwVvVurfpuz+XPuFvucclVQEyPqcpUkHR0ne1RQzPs0=,231781,465,SUPER JUNIOR,NaN,NaN,31.0
3,dKMBWoZyScdxSkihKG+Vf47nc18N9q4m58+b4e7dSSE=,273554,465,S.H.E,湯小康,徐世珍,3.0
4,W3bqWd3T+VeHFzHAUfARgW9AvVRaF4N5Yzm4Mr6Eo/o=,140329,726,貴族精選,Traditional,Traditional,52.0
5,kKJ2JNU5h8rphyW21ovC+RZU+yEHPM+3w85J37p7vEQ=,235520,864|857|850|843,貴族精選,Joe Hisaishi,Hayao Miyazaki,17.0
...,...,...,...,...,...,...,...
2296315,lg6rn7eV/ZNg0+P+x77kHUL7GDMfoL4eMtXxncseLNA=,20192,958,Catherine Collard,Robert Schumann (1810-1856),NaN,-1.0
2296316,nXi1lrSJe+gLoTTNky7If0mNPrIyCQCLwagwR6XopGU=,273391,465,紀文惠 (Justine Chi),NaN,NaN,3.0
2296317,9KxSvIjbJyJzfEVWnkMbgR6dyn6d54ot0N5FKyKqii8=,445172,1609,Various Artists,NaN,NaN,52.0
2296318,UO8Y2MR2sjOn2q/Tp8/lzZTGKmLEvwZ20oWanG4XnYc=,172669,465,Peter Paul & Mary,NaN,NaN,52.0


In [81]:
songs.groupby("language").size()

language
-1.0      639467
 3.0      106295
 10.0      15482
 17.0      92518
 24.0      41744
 31.0      39201
 38.0       2385
 45.0      14435
 52.0    1336694
 59.0       8098
dtype: int64

### Training and Testing

In [109]:
class Data():
    def __init__(self, path1, path2, songs, users):
        self.train = pd.read_csv(path1)
        self.test = pd.read_csv(path2)
        self.people2songs = self.train.groupby('msno')["song_id"].apply(list)
        self.songs2count = self.train.groupby('song_id').size()
        self.song2user = self.train.groupby("song_id")["msno"].apply(list)
        self.songs = songs
        self.users = users
        self.songs_set = set(self.songs['song_id'])
        self.all = pd.concat([self.train, self.test], axis=0, sort=False)
        self.people2songs_all = self.all.groupby('msno')["song_id"].apply(set)
        
def sample(positive_data, all_songs, people2songs):
    negative_dict = dict()
    for people, count in tqdm(positive_data.groupby('msno').size().items()):
        negative_dict[people] = random.sample(all_songs - people2songs[people], count)
    append_list = []
    for people, songs in negative_dict.items():
        for song in songs:
            append_list.append([people, song, 0])
    positive_matrix = np.hstack((positive_data.values[:, :2], np.ones((positive_data.shape[0], 1))))
    return np.vstack((positive_matrix, np.array(append_list)))

In [100]:
train_path = os.path.join(data_path, "train.csv")
test_path = os.path.join(data_path, "test.csv")
data = Data(train_path, test_path, songs, users=members)        

In [112]:
len(data.train)

7377418

In [111]:
len(data.test)

2556790

In [110]:
blend_test = sample(data.test, data.songs_set, data.people2songs_all)

515it [03:02,  2.34it/s]

KeyboardInterrupt: 

In [107]:
len(data.test.groupby('msno').size())

25131

In [20]:
# def split(data, ratio):
#     ntrain = int(len(data)*ratio)
#     return data[:ntrain], data[ntrain:]

In [21]:
data = Data(path1=os.path.join(data_path, 'train.csv'))
test = pd.read_csv(os.path.join(data_path, "test.csv"))

### Sample Negative Samples for Testing

In [ ]:
def model():
    def __init__(self, train):
        self.train = train
        
    def baseline(self):
        self.songs_count = self.train['song_id'].values_count
        vc = self.train['song_id'].value_counts()
        return set(vc[vc > len(self.train[])]['song_id'])